In [1]:
%load_ext autoreload
%autoreload 2

import os
os.environ['FILEREADER_CACHE'] = '1'

from openpilot.tools.lib.logreader import LogReader


class Routes:
  ZERO = 'e886087f430e7fe7/2024-02-06--18-40-35'
  ONE = 'e886087f430e7fe7/2024-02-06--18-46-09'
  TWO = 'e886087f430e7fe7/2024-02-06--18-51-31'
  THREE = 'e886087f430e7fe7/2024-02-06--18-58-28'
  # FOUR = 'e886087f430e7fe7/2024-02-06--19-00-24'
  # FIVE = 'e886087f430e7fe7/2024-02-06--19-01-21'


kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /mnt/c/Users/camer/AppData/Local/Programs/Microsoft VS Code


In [2]:
def process(route):
  rows = []

  initTime = None
  for event in LogReader(route):
    if not initTime:
      initTime = event.logMonoTime
    if event.which() != 'can':
      continue

    # time = (event.logMonoTime - initTime) / 1e9
    for can in event.can:
      # if can.dat.startswith(TESTER_PRESENT_REQUEST) or can.dat.startswith(SINGLE_FRAME_TESTER_PRESENT_REQUEST):
      #   print(f'{can.address:x} {can.dat.hex()} TESTER PRESENT')
      # if can.dat.startswith(TESTER_PRESENT_RESPONSE) or can.dat.startswith(SINGLE_FRAME_TESTER_PRESENT_RESPONSE):
      #   print(f'{can.address:x} {can.dat.hex()} TESTER PRESENT RESPONSE')
      if can.src >= 4:
        continue
      if can.address < 0x550:
        continue
      # if can.address in (ADDR, RX_ADDR):
      # print(f'{can.address:x} {can.dat.hex()}')
      rows.append(((can.src, can.address), can.dat))

  return rows

In [3]:
from collections import defaultdict

from notebooks.forscan.isotp import parse_iso_tp


msgs = {}
for route in (Routes.ZERO, Routes.ONE, Routes.TWO, Routes.THREE):
  msgs[route] = process(route)
  count = defaultdict(int)
  for (src, address), dat in msgs[route]:
    frame = parse_iso_tp(dat)
    if frame is not None:
      count[address] += 1
  print(route, count)

e886087f430e7fe7/2024-02-06--18-40-35 defaultdict(<class 'int'>, {2019: 6, 1892: 90, 1900: 112, 1888: 148, 1896: 131, 1847: 214, 1855: 124, 1840: 100, 1848: 106, 1798: 53, 1806: 104, 1783: 3, 1782: 3})
e886087f430e7fe7/2024-02-06--18-46-09 defaultdict(<class 'int'>, {1892: 52, 1900: 68, 1888: 270, 1896: 336, 1847: 18, 1855: 14, 1840: 430, 1848: 438, 1798: 187, 1806: 313})
e886087f430e7fe7/2024-02-06--18-51-31 defaultdict(<class 'int'>, {1798: 192, 1806: 220, 1840: 244, 1848: 143})
e886087f430e7fe7/2024-02-06--18-58-28 defaultdict(<class 'int'>, {1840: 1356, 1848: 678})


In [4]:
from notebooks.forscan.isotp import parse_iso_tp


iso_tp_frames = []
for addr, data in msgs[Routes.ZERO] + msgs[Routes.ONE] + msgs[Routes.TWO]:
  # if addr[1] not in (1840, 1848):
  #   continue
  # if addr[1] in (1430, 1438, 1459, 1461):
  #   continue
  # if addr not in (ADDR, RX_ADDR):
  #   continue

  frame = parse_iso_tp(data)
  if frame is not None:
    # print(f'{addr=}', frame)
    # if addr[0] in (0, 1):
    iso_tp_frames.append((addr[1], frame))
  # else:
  #   print(f'Failed to parse {addr=}', data.hex())

len(iso_tp_frames)

4119

In [5]:
from collections import defaultdict

def reconstruct_iso_tp_messages(frames):
  """
  Reconstructs complete ISO-TP payloads from a sequence of CAN messages.

  Args:
      msgs: A list of tuples (address, data), where address is the CAN address and
            data is the CAN message payload as bytes.

  Returns:
      A list of dictionaries representing the reconstructed messages, each containing:
          'addr': The CAN address where the message was received.
          'payload': The complete reassembled message payload as bytes.
  """

  reconstructed_msgs = []
  in_progress = defaultdict(list)  # To store partial frames of a message

  for addr, frame in frames:
    if frame.frame_type == 'single':
      reconstructed_msgs.append({'addr': addr, 'payload': frame.payload})

    elif frame.frame_type == 'first':
      # Start a new reconstruction - clear buffer in case any was in progress
      if len(in_progress[addr]):
        print(f'Warning: Discarding incomplete message from {addr:x}')
      in_progress[addr].clear() 
      in_progress[addr].append(frame)

    elif frame.frame_type == 'consecutive':
      if len(in_progress[addr]) == 0:
        print(f'Warning: Discarding orphaned consecutive frame from {addr:x}')
        continue

      in_progress[addr].append(frame)

      # Check if all segments are received based on expected size from First frame
      if sum(len(f.payload) for f in in_progress[addr]) >= in_progress[addr][0].size:
        payload = b''.join(f.payload for f in in_progress[addr])
        # Trim payload to expected size
        payload = payload[:in_progress[addr][0].size]
        assert len(payload) == in_progress[addr][0].size
        reconstructed_msgs.append({'addr': addr, 'payload': payload})
        in_progress[addr].clear()  # Message assembled - clear the buffer

    # Optionally handle potential FlowControlFrame if applicable, else it is discarded silently

  return reconstructed_msgs

reconstructed = reconstruct_iso_tp_messages(iso_tp_frames)
# for msg in reconstructed:
#   print(msg)

In [6]:
from notebooks.ford.ecu import FordEcu
from notebooks.forscan.uds import parse_uds

ADDRS = [
  FordEcu.AccessoryProtocolInterfaceModule,
  FordEcu.ImageProcessingModuleA,
  FordEcu.PowerSteeringControlModule,
]
ADDRS = ADDRS + [addr + 0x8 for addr in ADDRS]

for msg in reconstructed:
  if msg['addr'] not in ADDRS:
    continue
  uds = parse_uds(msg['payload'])
  if uds is None:
    print(f'Failed to parse {msg=}')
    continue
  print({ 'addr': msg['addr'], **uds })

{'addr': 1840, 'type': 'request', 'service': 'READ_DATA_BY_IDENTIFIER', 'data': '0200'}
{'addr': 1840, 'type': 'request', 'service': 'READ_DATA_BY_IDENTIFIER', 'data': '0200'}
{'addr': 1848, 'type': 'negative_response', 'data': '2231'}
{'addr': 1840, 'type': 'request', 'service': 'TESTER_PRESENT', 'data': '00'}
{'addr': 1840, 'type': 'request', 'service': 'TESTER_PRESENT', 'data': '00'}
{'addr': 1848, 'type': 'positive_response', 'service': 'TESTER_PRESENT', 'data': '00'}
{'addr': 1840, 'type': 'request', 'service': 'READ_DATA_BY_IDENTIFIER', 'data': 'f190'}
{'addr': 1840, 'type': 'request', 'service': 'READ_DATA_BY_IDENTIFIER', 'data': 'f190'}
{'addr': 1848, 'type': 'positive_response', 'service': 'READ_DATA_BY_IDENTIFIER', 'data': 'f1905746304e58584743484e4a52363937313200000000000000'}
{'addr': 1840, 'type': 'request', 'service': 'READ_DATA_BY_IDENTIFIER', 'data': 'f113'}
{'addr': 1840, 'type': 'request', 'service': 'READ_DATA_BY_IDENTIFIER', 'data': 'f113'}
{'addr': 1848, 'type': 'p